In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2


In [2]:
import pickle
import pandas as pd

In [3]:
YEAR: int= 2021
MONTH: int= 2
OUTPUT_FILE: str= "./output/ride_predictions_{YEAR:04d}_{MONTH:02d}.parquet"

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [5]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data(f'https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_{YEAR:4d}-{MONTH:02d}.parquet')
df['ride_id'] = f'{YEAR:04d}/{MONTH:02d}_' + df.index.astype('str')

In [7]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [8]:
y_pred.mean()

16.191691679979066

In [9]:
df_result = pd.DataFrame()
df_result["ride_id"] = df["ride_id"]
df_result["ride_prediction"] = y_pred

In [11]:
df_result.to_parquet(
    OUTPUT_FILE,
    engine='pyarrow',
    compression=None,
    index=False
)

In [12]:
type(y_pred)

numpy.ndarray